Dataframes to Parquet for Production and Popularity Evaluation in Spark

In [4]:
import pandas as pd
import numpy as np
import fastparquet

In [2]:
pip install fastparquet

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.6 MB 39.7 MB/s eta 0:00:01
     |████████████████████████████████| 12.2 MB 116.1 MB/s eta 0:00:01
  Using cached cramjam-2.6.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
     |████████████████████████████████| 17.1 MB 115.7 MB/s eta 0:00:01
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

pytorch-lightning 1.8.3.post1 requires PyYAML>=5.4, but you'll have pyyaml 5.3.1 which is incompatible.
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 9.7 MB 36.0 MB/s eta 0:00:01
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
df = pd.read_csv("merged_train.csv", header = None)
df_test = pd.read_csv("merged_test.csv", header = None)

In [7]:
df_test.columns = ["tracing_id", "zpid_hash", "feature1", "feature2", "feature3", "feature4", "feature5", "feature6", "feature7", "feature8","user_id_hash", "timestamp_session", "user_session_id", "order", "submit", "fav", "click"]

In [8]:
df.columns = ["tracing_id", "zpid_hash", "feature1", "feature2", "feature3", "feature4", "feature5", "feature6", "feature7", "feature8","user_id_hash", "timestamp_session", "user_session_id", "order", "submit", "fav", "click"]

In [9]:
df_test.click = df_test.click.replace({ True: 1, False: 0})
df_test.submit = df_test.submit.replace({True: 1, False: 0})
df_test.fav = df_test.fav.replace({True: 1, False: 0})

In [10]:
df.click = df.click.replace({ True: 1, False: 0})
df.submit = df.submit.replace({True: 1, False: 0})
df.fav = df.fav.replace({True: 1, False: 0})

Label Encoding Property Ids

In [11]:
df_merged = pd.concat([df, df_test])

In [12]:
labelencoder = LabelEncoder()
df_merged["zpid_encoded"]= labelencoder.fit_transform(df_merged["zpid_hash"])

In [13]:
df_test = df_merged[29359673:]

In [14]:
df = df_merged[:29359673]

Top 20

In [15]:
df_test["weighted_score"] = 0.5*df_test["fav"] + 0.2*df_test["click"] + df_test["submit"]

<ipython-input-15-fedfe7dbe10e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["weighted_score"] = 0.5*df_test["fav"] + 0.2*df_test["click"] + df_test["submit"]


In [16]:
df["weighted_score"] = 0.5*df["fav"] + 0.2*df["click"] + df["submit"]

<ipython-input-16-8a437d9ea019>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["weighted_score"] = 0.5*df["fav"] + 0.2*df["click"] + df["submit"]


In [17]:
top_20_weighted = df.groupby('zpid_encoded')['weighted_score'].sum().sort_values(ascending = False).head(20)

In [18]:
top_20_fav = df.groupby('zpid_encoded')['fav'].sum().sort_values(ascending = False).head(20)

In [19]:
top_20_click = df.groupby('zpid_encoded')['click'].sum().sort_values(ascending = False).head(20)

In [20]:
top_20_submit = df.groupby('zpid_encoded')['submit'].sum().sort_values(ascending = False).head(20)

In [21]:
top_20_weighted = top_20_weighted.to_frame().reset_index()
top_20_fav = top_20_fav.to_frame().reset_index()
top_20_submit = top_20_submit.to_frame().reset_index()
top_20_click = top_20_click.to_frame().reset_index()

In [22]:
top_20_weighted = top_20_weighted.drop('weighted_score', axis = 1)
top_20_fav = top_20_fav.drop('fav', axis = 1)
top_20_submit = top_20_submit.drop('submit', axis = 1)
top_20_click = top_20_click.drop('click', axis = 1)

In [23]:
top_20_weighted.to_parquet("top_20_weighted.parquet", engine='fastparquet')
top_20_click.to_parquet("top_20_click.parquet", engine='fastparquet')
top_20_fav.to_parquet("top_20_fav.parquet", engine='fastparquet')
top_20_submit.to_parquet("top_20_submit.parquet", engine='fastparquet')

Test

In [24]:
df_test_2= df_test.sort_values(by = ["tracing_id", "order"])

In [25]:
df_test_2 = df_test_2[df_test_2["order"] <=19] 

In [26]:
df_test_2 

,tracing_id,zpid_hash,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,user_id_hash,timestamp_session,user_session_id,order,submit,fav,click,zpid_encoded,weighted_score
3103616,62ca193dc36929bb30f091105b8e5bd1,8547e3c3f8e4987b028bdb26bdcbbad4f594b5ea,9418,6594,8848,9069,1212,1126,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,0,0,0,0,657630,0.0
3103612,62ca193dc36929bb30f091105b8e5bd1,004b8596022713dff74dc9df1c0d4e936b82022a,9418,475,8848,9069,11,127,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,1,0,0,0,1458,0.0
3103650,62ca193dc36929bb30f091105b8e5bd1,eacffeb4d791ae64fbb8959d445cb414b205df7c,9418,6594,8848,9069,2111,23,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,2,0,0,0,1000251,0.0
3103644,62ca193dc36929bb30f091105b8e5bd1,b79a7a68c1d71c9305317402023ddd86b1289fe4,9418,475,8848,9069,3507,1126,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,3,0,0,1,827100,0.2
3103648,62ca193dc36929bb30f091105b8e5bd1,5260c39a095eb64846e7c7d6b22c076da5268e2b,9418,6594,8848,9069,408,127,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,4,0,0,0,406755,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459778,62cb67f1265d7ed45bd56d5bf981af4a,159d41dd6b150c3a168fff68aba3521b051cffad,8611,77,1001,555,198,55,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,17,0,0,0,106148,0.0
459780,62cb67f1265d7ed45bd56d5bf981af4a,60d9a1377605e75f19dabea54cdce6e8239771,8611,62,1001,7777,27,880,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,18,0,0,0,477977,0.0
459767,62cb67f1265d7ed45bd56d5bf981af4a,14ffa478495819d6aee2a20aa41a754660ea4139,8611,62,1001,7777,158,55,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,19,0,0,0,103164,0.0
4123133,62cb67f319bef3b9f7b6c258aa7fc78c,b5cf95f407257181efde17f5c27e637f171102e2,9738,8750,6509,9581,610,5877,6398,6398,806275b220719de60849a5aeab92f6ee56f402ab,2022-07-10 23:59:47.119,9df1e8c1f1b742518d157119b54490d3_2022_07_10_23_1,0,0,0,1,818028,0.2


In [27]:
df_test_all = df_test_2[["tracing_id", "zpid_encoded"]]

In [28]:
df_test_all.to_parquet("df_test.parquet", engine='fastparquet')

Production

In [29]:
df_test

,tracing_id,zpid_hash,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,user_id_hash,timestamp_session,user_session_id,order,submit,fav,click,zpid_encoded,weighted_score
0,62ca195944401fed240b385fa47bd155,0c6bc6449d4d4820898868330d9c5517ca100921,4166,833,83,3333,83,166,1833,3,591451d586ff228f1766748cb9ed2ba92847c237,2022-07-10 00:12:09.977,1d6fdd41f1f74ed9b13a7c0de133ac11_2022_07_10_00_1,33,0,0,0,60908,0.0
1,62ca195944401fed240b385fa47bd155,f48d6ed4c5403495655995a3fd12f41841ec319,4166,833,83,3333,83,166,1833,3,591451d586ff228f1766748cb9ed2ba92847c237,2022-07-10 00:12:09.977,1d6fdd41f1f74ed9b13a7c0de133ac11_2022_07_10_00_1,3,0,0,0,1048382,0.0
2,62ca195944401fed240b385fa47bd155,1c45210433d89c96c20fbc333f45b72ad2de69f,4166,833,83,3333,675,166,1833,3,591451d586ff228f1766748cb9ed2ba92847c237,2022-07-10 00:12:09.977,1d6fdd41f1f74ed9b13a7c0de133ac11_2022_07_10_00_1,17,0,0,0,139089,0.0
3,62ca195944401fed240b385fa47bd155,656785af4f2359802b72f5df73affbc55176ea0b,4166,500,83,3333,83,375,1833,3,591451d586ff228f1766748cb9ed2ba92847c237,2022-07-10 00:12:09.977,1d6fdd41f1f74ed9b13a7c0de133ac11_2022_07_10_00_1,1,0,0,0,500374,0.0
4,62ca195944401fed240b385fa47bd155,f723dabd44f495359b60e657f3df4600f1640b2,4166,833,83,3333,83,166,1833,3,591451d586ff228f1766748cb9ed2ba92847c237,2022-07-10 00:12:09.977,1d6fdd41f1f74ed9b13a7c0de133ac11_2022_07_10_00_1,13,0,0,0,1061122,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12576028,62cb671bac41c164218b997cdd95a92a,c5ed3b7e0f9a0c7fb7c7a954fa7c4fc7957843d6,724,6896,6586,7241,4948,4982,6620,6586,4eef169b8a206fb50a0392cd528f8e7ce5ab834f,2022-07-10 23:56:11.441,33ffbeaefa3d4d168e1221a0b74b1d04_2022_07_10_23_1,0,0,0,0,897480,0.0
12576029,62cb671bac41c164218b997cdd95a92a,7fd3d4cdda78ac39a73d55363aa8511aee95ce5e,724,344,6586,7241,4948,8,6620,6586,4eef169b8a206fb50a0392cd528f8e7ce5ab834f,2022-07-10 23:56:11.441,33ffbeaefa3d4d168e1221a0b74b1d04_2022_07_10_23_1,1,0,0,0,630690,0.0
12576030,62cb671bac41c164218b997cdd95a92a,1703c8dfaaba798e61d9a221bf257ba1096f50d2,724,344,6586,689,4948,8,6620,6586,4eef169b8a206fb50a0392cd528f8e7ce5ab834f,2022-07-10 23:56:11.441,33ffbeaefa3d4d168e1221a0b74b1d04_2022_07_10_23_1,4,0,0,0,113001,0.0
12576031,62cb671bac41c164218b997cdd95a92a,3121985799f9b8c40be808c24a820b4f60722c48,724,344,6586,7241,34,8,6620,6586,4eef169b8a206fb50a0392cd528f8e7ce5ab834f,2022-07-10 23:56:11.441,33ffbeaefa3d4d168e1221a0b74b1d04_2022_07_10_23_1,3,0,0,1,242645,0.2


In [30]:
df_test_2 = df_test.sort_values(by = ["tracing_id", "order"])

In [31]:
df_test_2

,tracing_id,zpid_hash,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,user_id_hash,timestamp_session,user_session_id,order,submit,fav,click,zpid_encoded,weighted_score
3103616,62ca193dc36929bb30f091105b8e5bd1,8547e3c3f8e4987b028bdb26bdcbbad4f594b5ea,9418,6594,8848,9069,1212,1126,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,0,0,0,0,657630,0.0
3103612,62ca193dc36929bb30f091105b8e5bd1,004b8596022713dff74dc9df1c0d4e936b82022a,9418,475,8848,9069,11,127,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,1,0,0,0,1458,0.0
3103650,62ca193dc36929bb30f091105b8e5bd1,eacffeb4d791ae64fbb8959d445cb414b205df7c,9418,6594,8848,9069,2111,23,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,2,0,0,0,1000251,0.0
3103644,62ca193dc36929bb30f091105b8e5bd1,b79a7a68c1d71c9305317402023ddd86b1289fe4,9418,475,8848,9069,3507,1126,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,3,0,0,1,827100,0.2
3103648,62ca193dc36929bb30f091105b8e5bd1,5260c39a095eb64846e7c7d6b22c076da5268e2b,9418,6594,8848,9069,408,127,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,4,0,0,0,406755,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459772,62cb67f1265d7ed45bd56d5bf981af4a,497a9a9c80f5ef8975d31e269859be255422b3fb,8611,1903,1001,7777,158,55,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,23,0,0,0,362880,0.0
459773,62cb67f1265d7ed45bd56d5bf981af4a,43d757c42467a59829f164c8fede15972ac8a67b,8611,77,1001,555,27,55,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,24,0,0,0,334958,0.0
459779,62cb67f1265d7ed45bd56d5bf981af4a,5afc66bbc93cea5309c01b8444aa39997aa0fa69,8611,77,1001,555,1034,55,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,25,0,0,0,449124,0.0
4123133,62cb67f319bef3b9f7b6c258aa7fc78c,b5cf95f407257181efde17f5c27e637f171102e2,9738,8750,6509,9581,610,5877,6398,6398,806275b220719de60849a5aeab92f6ee56f402ab,2022-07-10 23:59:47.119,9df1e8c1f1b742518d157119b54490d3_2022_07_10_23_1,0,0,0,1,818028,0.2


In [32]:
df_test_2 = df_test_2[df_test_2["weighted_score"] >= 0.2 ]

In [33]:
df_test_2["user_preference"] = df_test_2.groupby(["tracing_id"])["weighted_score"].rank(method='first', ascending = False)

<ipython-input-33-8abcba6f459b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_2["user_preference"] = df_test_2.groupby(["tracing_id"])["weighted_score"].rank(method='first', ascending = False)


In [34]:
df_test_2

,tracing_id,zpid_hash,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,user_id_hash,timestamp_session,user_session_id,order,submit,fav,click,zpid_encoded,weighted_score,user_preference
3103644,62ca193dc36929bb30f091105b8e5bd1,b79a7a68c1d71c9305317402023ddd86b1289fe4,9418,475,8848,9069,3507,1126,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,3,0,0,1,827100,0.2,1.0
3103633,62ca193dc36929bb30f091105b8e5bd1,5db96e84277e533ca6d2c55fc8408c8fc3a0ecc0,9418,6594,8848,9069,1094,2431,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,6,0,0,1,462555,0.2,2.0
3103651,62ca193dc36929bb30f091105b8e5bd1,f0b2717e076ea055e72aa9b0d3bd7df9a7f10377,9418,6594,8848,9069,1094,1126,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,8,0,0,1,1029189,0.2,3.0
3103624,62ca193dc36929bb30f091105b8e5bd1,051cbec0834e7e4838f9590c9a352c31c8efafe7,9418,475,8848,9069,1094,1126,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,17,0,0,1,25116,0.2,4.0
3932249,62ca193dcfc19c8d05fb5d9ef28d608e,016c7f46d6bab115af55401145388b8666fc4496,6406,500,5695,5002,1361,1458,4729,4444,7082924959a1c1fcc5ae3658ad03e6dc5a7ab30b,2022-07-10 00:11:41.653,8ba80684eedd4444b91a9b8ed246dd44_2022_07_10_00_1,5,0,1,1,7016,0.7,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2142652,62cb67eab13051ce4cfff33a86fb243f,6f9c0dca93f0edf551bf1092176454da020f4059,8913,3218,21,2565,2981,43,43,21,9a23299c0f568c0fd5684c0c7eae8d612db3776d,2022-07-10 23:59:38.607,9e91f6d93b1649fb8e25d6ff48113486_2022_07_10_23_1,0,0,0,1,550808,0.2,1.0
197791,62cb67eeb312a541990e794692ce2a4a,1536719c1d09878082809afbad2eed6291e136,9638,6122,4,9619,1119,9,9,4,778da0d1bdc73575ab77a48c1c127c7413999f2c,2022-07-10 23:59:43.040,438efec6aaf744b8ba215e50fbec08fc_2022_07_10_23_4,2,0,0,1,104212,0.2,1.0
2335194,62cb67ef76d830cf4b0c83d553cd0043,bbfc79ec1a31a6f3cc62e119fbe791b4898a644a,8526,3351,7815,3968,1563,1284,839,7815,aaa47571d3307ffe2f4026e764c5184d375d8306,2022-07-10 23:59:43.097,f712e3eb4b784dc199a1f310212e6f80_2022_07_10_23_1,10,0,0,1,848655,0.2,1.0
459762,62cb67f1265d7ed45bd56d5bf981af4a,486ed025904d4328370388e4e6b967b3cdbd0fed,8611,3698,1001,7777,1383,55,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,0,0,0,1,357773,0.2,1.0


In [35]:
df_test_2 = df_test_2[df_test_2["user_preference"] <= 20] 

In [36]:
df_test_2 = df_test_2[["tracing_id","zpid_encoded"]]

In [37]:
df_test_2

,tracing_id,zpid_encoded
3103644,62ca193dc36929bb30f091105b8e5bd1,827100
3103633,62ca193dc36929bb30f091105b8e5bd1,462555
3103651,62ca193dc36929bb30f091105b8e5bd1,1029189
3103624,62ca193dc36929bb30f091105b8e5bd1,25116
3932249,62ca193dcfc19c8d05fb5d9ef28d608e,7016
...,...,...
2142652,62cb67eab13051ce4cfff33a86fb243f,550808
197791,62cb67eeb312a541990e794692ce2a4a,104212
2335194,62cb67ef76d830cf4b0c83d553cd0043,848655
459762,62cb67f1265d7ed45bd56d5bf981af4a,357773


In [38]:
df_test_2.to_parquet("df_test_weighted.parquet", engine='fastparquet')

In [39]:
df_test_2

,tracing_id,zpid_encoded
3103644,62ca193dc36929bb30f091105b8e5bd1,827100
3103633,62ca193dc36929bb30f091105b8e5bd1,462555
3103651,62ca193dc36929bb30f091105b8e5bd1,1029189
3103624,62ca193dc36929bb30f091105b8e5bd1,25116
3932249,62ca193dcfc19c8d05fb5d9ef28d608e,7016
...,...,...
2142652,62cb67eab13051ce4cfff33a86fb243f,550808
197791,62cb67eeb312a541990e794692ce2a4a,104212
2335194,62cb67ef76d830cf4b0c83d553cd0043,848655
459762,62cb67f1265d7ed45bd56d5bf981af4a,357773
